In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Mecab
!apt update && apt install -y \
    curl \
    file \
    git \
    libmecab-dev \
    make \
    mecab \
    mecab-ipadic-utf8 \
    swig \
    xz-utils 
!pip install mecab-python3

In [0]:
#CRF++
import os

filename_crfpp = 'crfpp.tar.gz'
!wget "https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ" \
    -O $filename_crfpp
!tar zxvf $filename_crfpp
!cd CRF++-0.58 && ./configure && make && make install

os.environ['LD_LIBRARY_PATH'] += ':/usr/local/lib' 

In [0]:
#CaboCha
# https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7SDd1Q1dUQkZQaUU よりダウンロード
filename_cabocha = 'cabocha-0.69.tar.bz2'
#!wget "$url_cabocha" -O $filename_cabocha
!tar -jxf $filename_cabocha
!cd cabocha-0.69 && ./configure --with-mecab-config=`which mecab-config` --with-charset=UTF8
!cd cabocha-0.69 && make && make check && make install
!cabocha --version

In [0]:
#cabocha with python
!cd cabocha-0.69/python && python setup.py build_ext && python setup.py install
!cd cabocha-0.69/python && ldconfig

# 第5章: 係り受け解析
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [0]:
!cabocha -f1 "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt" -o "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"

In [41]:
import CaboCha
import codecs

class Morph(object):

    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音

def generate_morph(cabocha_file):
    parser = CaboCha.Parser()
    morph_data = []
    with codecs.open(cabocha_file, "r", "utf-8") as in_f:
        sentence = []
        for line in in_f.readlines():
            words = line.split(' ')
            if words[0] == "*":
                continue
            elif words[0].strip() == "EOS":
                if len(sentence) > 0:
                    morph_data.append(sentence)
                    sentence = []
                continue
            all_morph = words[0].split(",")
            morph = Morph(all_morph[0].split("\t")[0], all_morph[6], all_morph[0].split("\t")[1], all_morph[1])
            sentence.append(morph)
    return morph_data

morph_result = generate_morph("/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha")
for m in morph_result[2]:
    print(m.__dict__.values())


dict_values(['名前', '名前', '名詞', '一般'])
dict_values(['は', 'は', '助詞', '係助詞'])
dict_values(['まだ', 'まだ', '副詞', '助詞類接続'])
dict_values(['無い', '無い', '形容詞', '自立'])
dict_values(['。', '。', '記号', '句点'])


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [44]:
import CaboCha
import codecs

import re


class Morph(object):
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(
            self.surface, self.base, self.pos, self.pos1)

class Chunk(object):
# * 文節番号 係り先番号(-1) 主辞/機能語の位置 係り関係のスコア
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []

    def __str__(self):
        surface = ''
        for morph in self.morphs:
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)

def generate_chunk(neko_file):
    '''
    係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す

    戻り値：
    1文のChunkクラスのリスト
    '''
    with open(neko_file) as file_parsed:
        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1
        for line in file_parsed:
            if line == 'EOS\n':
                if len(chunks) > 0:
                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()
                else:
                    yield []
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))
                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
            else:
                cols = line.split('\t')
                res_cols = cols[1].split(',')
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )
        raise StopIteration


neko_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"
for i, chunks in enumerate(generate_chunk(neko_file), 1):
    # 8文目を表示
    if i == 8:
        for j, chunk in enumerate(chunks):
            print('[{}]{}'.format(j, chunk))
        break

[0]吾輩は	srcs[]	dst[5]
[1]ここで	srcs[]	dst[2]
[2]始めて	srcs[1]	dst[3]
[3]人間という	srcs[2]	dst[4]
[4]ものを	srcs[3]	dst[5]
[5]見た。	srcs[0, 4]	dst[-1]


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [76]:
import CaboCha
import codecs

import re


class Morph(object):
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(
            self.surface, self.base, self.pos, self.pos1)

class Chunk(object):
# * 文節番号 係り先番号(-1) 主辞/機能語の位置 係り関係のスコア
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []

    def __str__(self):
        surface = ''
        for morph in self.morphs:
            if not len(self.srcs):
                return
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)


def generate_chunk(neko_file):
    '''
    係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す
    '''
    with open(neko_file) as file_parsed:
        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1
        for line in file_parsed:
            if line == 'EOS\n':
                if len(chunks) > 0:
                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()
                else:
                    yield []
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))
                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
            else:
                cols = line.split('\t')
                res_cols = cols[1].split(',')
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )

def generate_related_cabocha(neko_file):
    result = []
    for ind, chunk_data in enumerate(generate_chunk(neko_file)):
        chunk_list = []
        for c in chunk_data:
            surface = ""
            for morph in c.morphs:
                surface += morph.surface
            obj = {surface: {"dst": c.dst, "srcs": c.srcs}}
            chunk_list.append(obj)
        for clause in chunk_list:
            srcs = clause[list(clause.keys())[0]]["srcs"]
            if not len(srcs):
                continue
            for chunk_ind in srcs:
                src_word = re.sub(r"[。、]", "", list(chunk_list[chunk_ind].keys())[0])
                effected_word = re.sub(r"[。、]", "", list(clause.keys())[0])
                result.append("{}\t{}".format(src_word, effected_word))
    return result

neko_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"
for cnt, word in enumerate(generate_related_cabocha(neko_file)):
    # 量が多いので5行分だけ出力
    if cnt > 5:
        break
    print(word)

　	猫である
吾輩は	猫である
名前は	無い
まだ	無い
　どこで	生れたか
生れたか	つかぬ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ． 

In [82]:
import CaboCha
import codecs

import re


class Morph(object):
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(
            self.surface, self.base, self.pos, self.pos1)

class Chunk(object):
# * 文節番号 係り先番号(-1) 主辞/機能語の位置 係り関係のスコア
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []

    def __str__(self):
        surface = ''
        for morph in self.morphs:
            if not len(self.srcs):
                return
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)


def generate_chunk(neko_file):
    '''
    係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す
    '''
    with open(neko_file) as file_parsed:
        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1
        for line in file_parsed:
            if line == 'EOS\n':
                if len(chunks) > 0:
                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()
                else:
                    yield []
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))
                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
            else:
                cols = line.split('\t')
                res_cols = cols[1].split(',')
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )

def delete_panctuation(word):
    return re.sub(r"[、。]", "", word)

def generate_related_cabocha(neko_file):
    result = []
    for ind, chunk_data in enumerate(generate_chunk(neko_file)):
        chunk_list = []
        for c in chunk_data:
            surface = ""
            pos_list = []
            for morph in c.morphs:
                surface += morph.surface
                pos_list.append(morph.pos)
            obj = {surface: {"dst": c.dst, "srcs": c.srcs, "pos": pos_list}}
            chunk_list.append(obj)
        for clause in chunk_list:
            srcs = clause[list(clause.keys())[0]]["srcs"]
            pos = clause[list(clause.keys())[0]]["pos"]
            if not len(srcs):
                continue
            for chunk_ind in srcs:
                src_word = list(chunk_list[chunk_ind].keys())[0]
                effected_word = list(clause.keys())[0]
                if "名詞" in chunk_list[chunk_ind][src_word]["pos"] and "動詞" in pos:
                    # src:名詞 -> eff:動詞
                    result.append("{}\t{}".format(
                        delete_panctuation(src_word), delete_panctuation(effected_word)))
    return result

neko_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"
for cnt, word in enumerate(generate_related_cabocha(neko_file)):
    # 量が多いので5行分だけ出力
    if cnt > 5:
        break
    print(word)

　どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
ここで	始めて


## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [0]:
!pip install graphviz pydot pillow

In [0]:
import CaboCha
import codecs
from pydot import Dot, Node, Edge
from PIL import Image

import re


class Morph(object):
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(
            self.surface, self.base, self.pos, self.pos1)

class Chunk(object):
# * 文節番号 係り先番号(-1) 主辞/機能語の位置 係り関係のスコア
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []

    def __str__(self):
        surface = ''
        for morph in self.morphs:
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)

def generate_chunk(neko_file):
    '''
    係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す

    戻り値：
    1文のChunkクラスのリスト
    '''
    with open(neko_file) as file_parsed:
        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1
        for line in file_parsed:
            if line == 'EOS\n':
                if len(chunks) > 0:
                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()
                else:
                    yield []
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))
                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
            else:
                cols = line.split('\t')
                res_cols = cols[1].split(',')
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )

def create_graph(chunk_data):
    graph = Dot(graph_type='graph')
    nodes = []
    for i, chunk in enumerate(chunk_data):
        surface = ""
        for w in chunk.morphs:
            surface+=w.surface
        node = Node(f'"{i}"', label=surface)
        nodes.append(node)
        graph.add_node(node)
    for i, chunk in enumerate(chunk_data):
        if chunk.dst == -1:
            continue
        node_src = nodes[i]
        node_dst = nodes[chunk.dst]
        edge = Edge(node_src, node_dst)
        graph.add_edge(edge)
    return graph


neko_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"
for i, chunks in enumerate(generate_chunk(neko_file), 1):
    # 8文目を表示
    if i == 8:
        graph = create_graph(chunks)
        graph.write_png('q44.png')
        Image.open('q44.png').show()

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．


*   動詞を含む文節において，最左の動詞の基本形を述語とする
*   述語に係る助詞を格とする
*   述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で
見る    は を
```

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．



*   コーパス中で頻出する述語と格パターンの組み合わせ
*   「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）






In [0]:
# 要件
# 述語の格を知りたい
# 動詞＝述語と動詞に係っている文節の助詞を格
# 出力→述語 \t 格
# 述語：動詞を含む文節における動詞の基本形(surface)
# 格：述語に係る助詞
# 　　※助詞（文節）が複数ある場合、すべての助詞をスペース区切りで辞書順に並べる


In [0]:
import CaboCha
import codecs

import re


class Morph(object):
# 表層形 （Tab区切り）, 品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

    def __str__(self):
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'.format(
            self.surface, self.base, self.pos, self.pos1)

class Chunk(object):
# * 文節番号 係り先番号(-1) 主辞/機能語の位置 係り関係のスコア
    def __init__(self):
        self.morphs = []
        self.dst = -1
        self.srcs = []

    def __str__(self):
        surface = ''
        for morph in self.morphs:
            if not len(self.srcs):
                return
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)


def generate_chunk(neko_file):
    '''
    係り受け解析結果を順次読み込んで、
    1文ずつChunkクラスのリストを返す
    '''
    with open(neko_file) as file_parsed:
        chunks = dict()     # idxをkeyにChunkを格納
        idx = -1
        for line in file_parsed:
            if line == 'EOS\n':
                if len(chunks) > 0:
                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]
                    chunks.clear()
                else:
                    yield []
            elif line[0] == '*':
                cols = line.split(' ')
                idx = int(cols[1])
                dst = int(re.search(r'(.*?)D', cols[2]).group(1))
                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst
                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx)
            else:
                cols = line.split('\t')
                res_cols = cols[1].split(',')
                chunks[idx].morphs.append(
                    Morph(
                        cols[0],        # surface
                        res_cols[6],    # base
                        res_cols[0],    # pos
                        res_cols[1]     # pos1
                    )
                )

def delete_panctuation(word):
    return re.sub(r"[、。]", "", word)

def generate_verb_cases(neko_file):
    result = []
    for ind, chunk_data in enumerate(generate_chunk(neko_file)):
        chunk_list = []
        for c in chunk_data:
            surface = ""
            pos_list = []
            for morph in c.morphs:
                surface += morph.surface
                pos_list.append(morph.pos)
            obj = {surface: {"dst": c.dst, "srcs": c.srcs, "pos": pos_list, "morphs": c.morphs}}
            chunk_list.append(obj)
        for clause in chunk_list:
            pos = clause[list(clause.keys())[0]]["pos"]
            srcs = clause[list(clause.keys())[0]]["srcs"]
                
            for chunk_ind in srcs:
                if "動詞" not in pos:
                    continue
                for m_data in clause[list(clause.keys())[0]]["morphs"]:
                    if m_data.pos == "動詞":
                        verb = m_data.base
                        break
                for s in chunk_list[chunk_ind][list(chunk_list[chunk_ind].keys())[0]]["morphs"]:
                    if s.pos != "助詞":
                        continue
                    result.append("{}\t{}".format(
                        delete_panctuation(verb), delete_panctuation(s.surface)))
    return result

neko_file = "/content/drive/My Drive/Colab Notebooks/Python100本ノック/neko.txt.cabocha"
verb_file = "verb_list.txt"
for line in generate_verb_cases(neko_file):
    with codecs.open(verb_file, "a", "utf-8") as f:
        f.write(line + "\n")

In [54]:
# コーパス中で頻出する述語と格パターンの組み合わせ
! sort verb_list.txt | uniq -c | sort -nr | head -1
# 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）
! grep "する" verb_list.txt | uniq -c | sort -nr | head -1
! grep "見る" verb_list.txt | uniq -c | sort -nr | head -1
! grep "与える" verb_list.txt | uniq -c | sort -nr | head -1

  10405 つく	か
     53 する	で
     56 見る	て
     35 与える	に


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

*   項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
*   述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

```
始める  で      ここで
見る    は を   吾輩は ものを
```